In [1]:
from mpi4py import MPI
from dolfinx import mesh
import numpy as np
from dolfinx import fem
import ufl
from dolfinx import default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import pyvista
from dolfinx import plot

In [ ]:
# A unit mesh of 16*16 quadrilateral cells: Dicreticise the domain into finite elements
N=16
domain = mesh.create_unit_square(MPI.COMM_WORLD, N, N, mesh.CellType.quadrilateral)

# Finite element function space of Langrange polynomials of degree 2
V = fem.functionspace(domain, ("Lagrange", 2))

# Dirichlets BC iterpolated on boundary: Homogeneous
uD = fem.Function(V)
uD.interpolate(lambda x: np.zeros_like(x[0]))

# Mapping cells to facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

# Mapping BC 
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

# Defining Trial and Test fucntions 
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Parameters with Constant source term over domain
epsilon=0.001
b=fem.Constant(domain, np.array([1.0, 0.0], dtype=default_scalar_type))
f = fem.Constant(domain, default_scalar_type(1.0))

#Artificial Diffusion
h = ufl.CellDiameter(domain)
bmag = ufl.sqrt(ufl.dot(b, b))
delta = 0.5 * h / bmag
# η (eta) is the non-linear viscosity (Glen's Law)
# v_x is the trial function, phi is the test function
a = (2 * eta * ufl.dot(v_x, 0) * ufl.dot(phi, 0) + 0.5 * eta * ufl.dot(v_x, 1) * ufl.dot(phi, 1)) * dx
L = rho * g * ufl.dot(h, 0) * phi * dx

a_ad = delta * ufl.dot(b, ufl.grad(u)) * ufl.dot(b, ufl.grad(v)) * ufl.dx
L_ad = delta * f * ufl.dot(b, ufl.grad(v)) * ufl.dx



# Writing the problem in variational: Bilinear and Linear
a = epsilon* ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx+ufl.dot(b, ufl.grad(u)) * v * ufl.dx
L = f * v * ufl.dx  


# Defining the problem: Assembly
problem = LinearProblem(
    a+a_ad,
    L+L_ad,
    bcs=[bc],
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
    petsc_options_prefix="Poisson",
)

#Solving the problem
uh = problem.solve()

In [ ]:
#Calculating the error:Interpolate exact solutio in function space
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2, name="u_exact")
uex.interpolate(lambda x: np.zeros_like(x[0]))

#L2 norm error: 
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

#Maximum error at any DOF:
error_max = numpy.max(numpy.abs(uD.x.array - uh.x.array))

# Only print the error on one process
if domain.comm.rank == 0:  
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

In [ ]:
#Visualise: Mesh
print(pyvista.global_theme.jupyter_backend)


domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesha.png")

plotter.screenshot("fundamentals_mesha.png")
plotter.close()

#Visualize: the solution, DOF in 2D
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)

u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True, show_scalar_bar=True, scalar_bar_args={ 'vertical': True})
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

u_plotter.screenshot("solutiona.png")
u_plotter.close()
#Visualise:  the solution in 3D, Warped
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True, scalar_bar_args={ 'vertical': True})
if not pyvista.OFF_SCREEN:
    plotter2.show()

plotter2.screenshot("warped_solutiona.png")
plotter2.close() # Clean up memory